In [1]:
import os
import numpy as np
import torch
import warnings

warnings.filterwarnings("ignore")
if os.getcwd()[-8:]=='examples':
    os.chdir('..')

os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Possible GPUS

In [2]:
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import importlib

from opacus import PrivacyEngine
from opacus.validators import ModuleValidator

import src
import src.trainer as tr

### Settings

In [3]:
#### FOR DP
MAX_GRAD_NORM = 0.1
EPSILON = 1.0
DELTA = 1e-5

#### FOR TRAINING
LR = 2
EPOCHS = 40
BATCH_SIZE = 2048
MAX_PHYSICAL_BATCH_SIZE = 256

#### FOR SAVE
PATH = "./saved/"
NAME = "MNIST_DPSGD_GNResnet_EPS1"
SAVE_PATH = PATH + NAME

### FOR MODELING
MODEL_NAME = "ResNet10"
DATA = "MNIST"
NORMALIZE = {'mean':[0.1307],
             'std':[0.3081]}
N_CLASSES = 10

### Data loader

In [4]:
data = src.Datasets(data_name=DATA, train_transform = transforms.ToTensor())
train_loader, test_loader = data.get_loader(batch_size=BATCH_SIZE, drop_last_train=False, num_workers=16)

Data Loaded!
Train Data Length : 60000
Test Data Length : 10000


### Model & Optimizer

In [5]:
#### Load model
model = src.utils.load_model(model_name=MODEL_NAME, n_classes=N_CLASSES).cuda() # Load model
model = ModuleValidator.fix(model)
ModuleValidator.validate(model, strict=False)

pytorch_total_params = sum(p.numel() for p in model.parameters())

print("model params: {:.4f}M".format(pytorch_total_params/1000000))

#### Define optimizer
optimizer = torch.optim.SGD(model.parameters(),lr=LR, momentum=0.9)

ResNet10 is loaded.
model params: 4.9021M


### Load PrivacyEngine from Opacus

In [6]:
privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)

optimizer.target_epsilon = EPSILON
optimizer.target_delta = DELTA

print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")
# 2.0664

rmodel = src.RobModel(model, n_classes=N_CLASSES, normalize=NORMALIZE).cuda()

Using sigma=4.8046875 and C=0.1


### Start Training

In [7]:
importlib.reload(tr)

trainer = tr.DpTrainer(NAME,rmodel)
trainer.max_physical_batch_size = MAX_PHYSICAL_BATCH_SIZE
trainer.record_rob(train_loader, test_loader)

In [8]:
trainer.fit(train_loader=train_loader, max_epoch=EPOCHS, start_epoch=0,
            optimizer=optimizer,
            scheduler=None, scheduler_type="Epoch",
            minimizer=None,
            save_path=SAVE_PATH, save_best={"Clean(Val)":"HB"},
            save_type=None, save_overwrite=True, record_type="Epoch")

Warning! Save file(s) will be overwritten:./saved/MNIST_DPSGD_GNResnet_EPS1
Warning! Save file(s) will be overwritten:./saved/MNIST_DPSGD_GNResnet_EPS1/last.pth
Warning! Save file(s) will be overwritten:./saved/MNIST_DPSGD_GNResnet_EPS1/best.pth
Warning! Save file(s) will be overwritten:./saved/MNIST_DPSGD_GNResnet_EPS1/record.csv
Warning! Save file(s) will be overwritten:./saved/MNIST_DPSGD_GNResnet_EPS1/summary.txt
[MNIST_DPSGD_GNResnet_EPS1]
Training Information.
-Epochs: 40
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 2
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
-Scheduler: None
-Save Path: ./saved/MNIST_DPSGD_GNResnet_EPS1
-Save Type: None
-Record Type: Epoch
-Device: cuda:0
--------------------------------------------------------
Epoch   CALoss   Clean(Tr)   Clean(Val)   lr   Sec/it   
1       1.9154   56.9618     56.8900      2    0.1125   
--------------------------------------------------------
2       1.2955   64.9519     64.5800      2    0.1

### Evaluation

In [9]:
rmodel.load_dict(PATH+NAME+'/last.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 40), ('CALoss', 0.22119358095748365), ('Clean(Tr)', 94.83911861353438), ('Clean(Val)', 95.22), ('lr', 2)])


95.22

In [10]:
rmodel.load_dict(PATH+NAME+'/best.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 38), ('CALoss', 0.2203147996684559), ('Clean(Tr)', 94.58182364629076), ('Clean(Val)', 95.3), ('lr', 2)])


95.3